In [1]:
import os
os.getcwd()

'/home/andreas/Documents/SINTEF_code_review/cidemod/Checks/Thermal'

In [2]:
from cideMOD import (
    CellParser,
    Problem,
    SolverCrashed,
    Trigger,
    ModelOptions,
)

import numpy as np
import matplotlib.pyplot as plt

In [3]:
#Define Chen path
overwrite = True
case = "Xu_2015"
data_path = "../Data/data_{}".format(case)
params = "params.json"

C_rate = -1
I_app = -5 #C_rate * problem.Q
t_f = 3600 /abs(C_rate)*1.25
v_min = Trigger(2.5, "v")

Define model options, we turn on thermal in 4D

In [4]:
i=8
model_options = ModelOptions(mode='P4D', solve_thermal=True, clean_on_exit=False,N_x=i,N_y=i,N_z=i)
cell = CellParser(params, data_path=data_path)

Capacidad Anodo: 0.10172665834785263
Capacidad Catodo: 0.1695044914659747
Capacidad Celda: 0.10172665834785263


Setup of the problem

In [5]:
problem= Problem(cell, model_options)
problem.set_cell_state(1, 273 + 25, 273 + 25)
problem.setup() #OBS Cylinder reads thr cylinder_3.xml file in the LOCAL repo


Building mesh for P4D problem with 5 components and 2560 nodes.
Finished building mesh
Building problem setup
 - Build cell parameters - Done 
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.

In [ ]:
status = problem.solve_ie(i_app=I_app, t_f=t_f, store_delay=10, adaptive=True, triggers=[v_min])